In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import ShortType, TimestampType
from delta.tables import DeltaTable
from datetime import datetime

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "base_atendentes"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_atendente"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

timestamp_atual = datetime.now()

In [0]:
try:
    df_bronze = spark.read.table(nome_tabela_origem)

    df_silver = df_bronze \
        .select(
            F.col("id_atendente").alias("cd_atendente"),
            F.trim(F.col("nome_atendente")).alias("nm_atendente"),
            F.col("nivel_atendimento").cast(ShortType()).alias("nu_nivel"),
            F.lit(timestamp_atual).cast(TimestampType()).alias("dt_ingestion"),
            F.col("origem").alias("dc_origem")
        ) \
        .filter(F.col("id_atendente").isNotNull()) \
        .dropDuplicates(["cd_atendente"])

    delta_table = DeltaTable.forName(spark, nome_tabela_destino)

    delta_table.alias("destino").merge(
        df_silver.alias("origem"),
        "destino.cd_atendente = origem.cd_atendente"
    ).whenMatchedUpdateAll(
    ).whenNotMatchedInsertAll(
    ).execute()

    print(f"✅ Tabela {nome_tabela_destino} carregada com sucesso")

except Exception as e:
    print(f"❌ Falha no processamento: {e}")
    raise